In [1]:
import pandas as pd
import numpy as np
from google.cloud import storage

import datetime as dt
from datetime import datetime
from pytz import timezone
pd.options.display.max_colwidth = 10000
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import uuid

(20413, 7)

In [ ]:
client = storage.Client()
bucket=client.get_bucket('aiadatascience')
blob = storage.Blob('genetics/uniprotFilteredOrganismHuman9606.tab',bucket)
with open('uniprotFilteredOrganismHuman9606.tab', 'wb') as file_obj:
    blob.download_to_file(file_obj)

In [ ]:
df_U=pd.read_csv('uniprotFilteredOrganismHuman9606.tab',sep='\t', header=0, \
               names=['Entry_U','GeneName','ProteinName_U','Organism_U','Entryname_U', \
                      'NaturalVariant_U','InvolmentInDisease_U'], \
               dtype={'Entry_U':object,'Entryname_U':object,'ProteinName_U':object, \
                      'GeneName':object,'Organism_U':object,'NaturalVariant_U':object,\
                      'Mutagenesis_U':object})

df_U=df_U[['GeneName','Entry_U','ProteinName_U','Organism_U','Entryname_U',\
           'NaturalVariant_U','InvolmentInDisease_U']]
df_U.shape

## Number of entires in above Uniprot list: 20,413 with 7 columns - dt. Jan 27 2019 (rerun Apr 9th)
### We only started with 7 columns so that the uniprot list is managable.

In [2]:
#df_U[df_U.Entry_U=='Q93074'] #df_U[df_U.GeneName=='MED12']
df_U[df_U.Entry_U=='Q96QU1']
#we can see that GeneName is comprising of more than one names. We are only considering the 1st Name as the GeneName

,GeneName,Entry_U,ProteinName_U,Organism_U,Entryname_U,NaturalVariant_U,InvolmentInDisease_U
9,PCDH15 USH1F,Q96QU1,Protocadherin-15,Homo sapiens (Human),PCD15_HUMAN,"VARIANT 19 19 S -> A (in dbSNP:rs11004439). {ECO:0000269|PubMed:16369489, ECO:0000269|PubMed:22815625}. /FTId=VAR_028289.; VARIANT 134 134 R -> G (in DFNB23; dbSNP:rs137853003). {ECO:0000269|PubMed:14570705, ECO:0000269|PubMed:18719945}. /FTId=VAR_024035.; VARIANT 134 134 R -> Q (in USH1F; dbSNP:rs767966376). {ECO:0000269|PubMed:22815625}. /FTId=VAR_071696.; VARIANT 174 174 N -> S (in dbSNP:rs145037203). {ECO:0000269|PubMed:22815625}. /FTId=VAR_071697.; VARIANT 178 178 D -> G (in USH1DF). {ECO:0000269|PubMed:18719945}. /FTId=VAR_069297.; VARIANT 262 262 G -> D (in DFNB23; dbSNP:rs137853002). {ECO:0000269|PubMed:14570705}. /FTId=VAR_024036.; VARIANT 380 380 G -> S (in dbSNP:rs10825269). {ECO:0000269|PubMed:22815625}. /FTId=VAR_028290.; VARIANT 435 435 D -> A (in dbSNP:rs4935502). {ECO:0000269|PubMed:17974005, ECO:0000269|PubMed:22815625}. /FTId=VAR_028291.; VARIANT 929 929 R -> Q (in dbSNP:rs2135720). {ECO:0000269|PubMed:11487575, ECO:0000269|PubMed:17974005, ECO:0000269|PubMed:22815625}. /FTId=VAR_028292.; VARIANT 1273 1273 R -> S (in dbSNP:rs111033363). {ECO:0000269|PubMed:22815625}. /FTId=VAR_071698.; VARIANT 1342 1342 Q -> K (in USH1F; dbSNP:rs61731387). {ECO:0000269|PubMed:15660226}. /FTId=VAR_024037.; VARIANT 1576 1955 Missing (in DFNB23). {ECO:0000269|PubMed:28281779}. /FTId=VAR_079507.; VARIANT 1867 1867 Missing (in USH1F). {ECO:0000269|PubMed:15660226}. /FTId=VAR_024038.","DISEASE: Usher syndrome 1F (USH1F) [MIM:602083]: USH is a genetically heterogeneous condition characterized by the association of retinitis pigmentosa with sensorineural deafness. Age at onset and differences in auditory and vestibular function distinguish Usher syndrome type 1 (USH1), Usher syndrome type 2 (USH2) and Usher syndrome type 3 (USH3). USH1 is characterized by profound congenital sensorineural deafness, absent vestibular function and prepubertal onset of progressive retinitis pigmentosa leading to blindness. {ECO:0000269|PubMed:15660226, ECO:0000269|PubMed:22815625}. Note=The disease is caused by mutations affecting the gene represented in this entry.; DISEASE: Usher syndrome 1D/F (USH1DF) [MIM:601067]: USH1DF patients are heterozygous for mutations in CDH23 and PCDH15, indicating a digenic inheritance pattern. {ECO:0000269|PubMed:15537665, ECO:0000269|PubMed:18719945}. Note=The disease is caused by mutations affecting distinct genetic loci, including the gene represented in this entry.; DISEASE: Deafness, autosomal recessive, 23 (DFNB23) [MIM:609533]: A form of non-syndromic sensorineural hearing loss. Sensorineural deafness results from damage to the neural receptors of the inner ear, the nerve pathways to the brain, or the area of the brain that receives sound information. {ECO:0000269|PubMed:14570705, ECO:0000269|PubMed:18719945, ECO:0000269|PubMed:28281779}. Note=The disease is caused by mutations affecting the gene represented in this entry."


#### Removing entries from Uniprot file without a gene name and considering only the 1st gene name for processing

In [3]:
df_U=df_U[df_U.GeneName.notnull()] #Removing entries from Uniprot file without a gene name.
df_U['GeneName']=df_U['GeneName'].apply(lambda x:x.split(' ')[0]) 
df_U.shape

(20262, 7)

## Number of entires in above Uniprot list: 20,262 with 7 columns

# Stage II Processing
### Extracting Natural Variant,dbSNP, Variant Position, PubMed,FromTo,MIM and Disease information

In [4]:
df_U_variant=df_U[['Entry_U','NaturalVariant_U']][df_U.NaturalVariant_U.notnull()]
df_U_disease=df_U[['Entry_U','InvolmentInDisease_U']][df_U.InvolmentInDisease_U.notnull()]

In [5]:
# This function extracts disease information from the involment in disease column.
import re
lst__D=[]
def findDisease(row):
    for x in row['InvolmentInDisease_U'].split('.;'):
        #print(x)
        #print('++++++++++++++++++++++')
        PubMed=''
        mim_=''
        disease_=''
        
        aa=[]
        search_pattern='(?s)DISEASE:(.*?)(?=\[)'
        aa=re.findall(search_pattern,x)            
        if len(aa)!=0:
            disease_=aa[0].strip()
        else:
            #DISEASE: Note=Defects in ATM contribute to B-cell non-Hodgkin lymphomas (BNHL), 
            #including mantle cell lymphoma (MCL).
            #DISEASE: Note=A chromosomal aberration involving BRAF is 
            #found in pilocytic astrocytomas. 
            #A tandem duplication of 2 Mb at 7q34 leads to the expression of a 
            #KIAA1549-BRAF fusion protein with a 
            #constitutive kinase activity and inducing cell transformation. 
            #{ECO:0000269|PubMed:18974108}
                
            search_pattern='(?:^|\W\()[A-Z]*(?:$|\))'
            aa=re.findall(search_pattern,x)
            disease_=' &'.join(aa).strip().replace('(','').replace(')','')
            
        aa=[]
        search_pattern="(?s)MIM:(.*?)(?=])"
        aa=re.findall(search_pattern,x)            
        if len(aa)!=0:
            mim_=aa[0]
            
        aa=[]
        search_pattern="(?s):(.*?)(?={)"
        aa=re.findall(search_pattern,x)
        #print(aa)
        if len(aa)!=0:
            desc_=aa[0]
            
        aa=[]
        search_pattern='(?s){(.*?)(?=})'
        aa=re.findall(search_pattern,x)
        #print(aa)
        if len(aa)!=0:
            PubMed=[]
            for i in aa[0].split(','):
                if '|' in i:                    
                    PubMed.append(i.strip().split('|')[1][7:])
                else:
                    PubMed.append(i)
            lst__D.append([row['Entry_U'],disease_,mim_,desc_,','.join(PubMed)])
            
    return(1)

In [6]:
#this function extracts variant information from Natural Variant column
import re
lst__V=[]
def findVariant(row):        
    for x in row['NaturalVariant_U'].split('.;'):
        dbSNP=''
        FromTo=''
        Variant=''
        PubMed=''
        aa=[]
        part1=''
        part2=''
        Muatation=''
        part1=x.strip().split('/')[0].strip()
        if len(x.strip().split('/'))==2:
            part2=x.strip().split('/')[1][5:].replace('.','')
        
        search_pattern='(?s)dbSNP:(.*?)(?=\).)'
        aa=re.findall(search_pattern,part1)            
        if len(aa)!=0:
            dbSNP=aa[0]
        
        aa=[]
        search_pattern='(?s)VARIANT(.*?)(?=\()'
        aa=re.findall(search_pattern,part1)            
        if len(aa)!=0:            
            if len(aa[0].strip().split(' '))==5:
                #print('---->'.join(aa))
                FromTo=aa[0].strip().split(' ')[0]+'-'+aa[0].strip().split(' ')[1]
                Variant=aa[0].strip().split(' ')[2]+aa[0].strip().split(' ')[3]+aa[0].strip().split(' ')[4]
                #print(FromTo,Variant)
            else:
                FromTo=aa[0].strip().split(' ')[0]+'-'+aa[0].strip().split(' ')[1]
                Variant=aa[0].strip().split(' ')[2] #Missing variant information??
            Muatation=aa[0].strip()
                            
        search_pattern='(?s){(.*?)(?=})'
        aa=re.findall(search_pattern,part1)
        ##print(aa)
        if len(aa)!=0:
            PubMed=[]
            #print(aa[0].split(','))
            for i in aa[0].split(','):                
                if '|' in i:
                    #print(i.strip().split('|')[1][7:])
                    PubMed.append(i.strip().split('|')[1][7:])
                else:
                    PubMed.append(i)
            lst__V.append([row['Entry_U'],part2,'Uniprot','Substitution-Missense',dbSNP, \
                           Muatation,','.join(PubMed)])
            
    return (1)

In [ ]:
#_=df_U_variant[df_U_variant.Entry_U=='Q96QU1'].apply(lambda row: findVariant(row),axis=1)
#lst__V
#_=df_U_disease[df_U_disease.Entry_U=='Q96QU1'].apply(lambda row: findDisease(row),axis=1)
#lst__D

In [7]:
#appling the functions
_=df_U_variant.apply(lambda row: findVariant(row),axis=1)
_=df_U_disease.apply(lambda row: findDisease(row),axis=1)

In [8]:
#cpnverting the list object to dataframe
_df_V = pd.DataFrame(data=lst__V,columns=['Entry_U','VariantID_V','VariantSource_U',\
                                          'MutationDescription_V','dbSNP_V','Mutation_V','PubMed_V'])
_df_D = pd.DataFrame(data=lst__D,columns=['Entry_U','Disease_D','MIM_D','Desc_D','PubMed_D'])

In [9]:
_df_V[_df_V.Entry_U=='Q93074']

,Entry_U,VariantID_V,VariantSource_U,MutationDescription_V,dbSNP_V,Mutation_V,PubMed_V
47866,Q93074,VAR_033112,Uniprot,Substitution-Missense,rs80338758,961 961 R -> W,17334363
47867,Q93074,VAR_037534,Uniprot,Substitution-Missense,rs80338759,1007 1007 N -> S,17369503
47868,Q93074,VAR_069770,Uniprot,Substitution-Missense,rs387907360,1148 1148 R -> H,23395478
47869,Q93074,VAR_069771,Uniprot,Substitution-Missense,rs387907361,1165 1165 S -> P,23395478
47870,Q93074,VAR_046672,Uniprot,Substitution-Missense,rs1139013,1392 1392 Q -> R,"10198638,8724849"
47871,Q93074,VAR_069772,Uniprot,Substitution-Missense,rs387907362,1729 1729 H -> N,23395478
47872,Q93074,VAR_074018,Uniprot,Substitution-Missense,rs879255528,1974 1974 Q -> H,26273451


In [10]:
df_U_variant[df_U_variant.Entry_U=='Q93074']

,Entry_U,NaturalVariant_U
17783,Q93074,"VARIANT 961 961 R -> W (in OKS; dbSNP:rs80338758). {ECO:0000269|PubMed:17334363}. /FTId=VAR_033112.; VARIANT 1007 1007 N -> S (in LUJFRYS; dbSNP:rs80338759). {ECO:0000269|PubMed:17369503}. /FTId=VAR_037534.; VARIANT 1148 1148 R -> H (in OHDOX; dbSNP:rs387907360). {ECO:0000269|PubMed:23395478}. /FTId=VAR_069770.; VARIANT 1165 1165 S -> P (in OHDOX; dbSNP:rs387907361). {ECO:0000269|PubMed:23395478}. /FTId=VAR_069771.; VARIANT 1392 1392 Q -> R (in dbSNP:rs1139013). {ECO:0000269|PubMed:10198638, ECO:0000269|PubMed:8724849}. /FTId=VAR_046672.; VARIANT 1729 1729 H -> N (in OHDOX; dbSNP:rs387907362). {ECO:0000269|PubMed:23395478}. /FTId=VAR_069772.; VARIANT 1974 1974 Q -> H (found in a family with X-linked intellectual disability; unknown pathological significance; dbSNP:rs879255528). {ECO:0000269|PubMed:26273451}. /FTId=VAR_074018."


In [11]:
_df_D[_df_D.Entry_U=='Q03164']

,Entry_U,Disease_D,MIM_D,Desc_D,PubMed_D
2840,Q03164,Wiedemann-Steiner syndrome (WDSTS),605130,"Wiedemann-Steiner syndrome (WDSTS) [MIM:605130]: A syndrome characterized by hairy elbows (hypertrichosis cubiti), intellectual disability, a distinctive facial appearance, and short stature. Facial characteristics include long eyelashes, thick or arched eyebrows with a lateral flare, and downslanting and vertically narrow palpebral fissures.",22795537
2841,Q03164,,,"Note=Chromosomal aberrations involving KMT2A are a cause of acute leukemias. Translocation t(1;11)(q21;q23) with MLLT11/AF1Q; translocation t(3;11)(p21;q23) with NCKIPSD/AF3p21; translocation t(3,11)(q25,q23) with GMPS; translocation t(4;11)(q21;q23) with AFF1/MLLT2/AF4; insertion ins(5;11)(q31;q13q23) with AFF4/AF5Q31; translocation t(5;11)(q12;q23) with AF5-alpha/CENPK; translocation t(6;11)(q27;q23) with AFDN; translocation t(9;11)(p22;q23) with MLLT3/AF9; translocation t(10;11)(p11.2;q23) with ABI1; translocation t(10;11)(p12;q23) with MLLT10/AF10; t(11;15)(q23;q14) with KNL1 and ZFYVE19; translocation t(11;17)(q23;q21) with MLLT6/AF17; translocation t(11;19)(q23;p13.3) with ELL; translocation t(11;19)(q23;p13.3) with MLLT1/ENL; translocation t(11;19)(q23;p23) with GAS7; translocation t(X;11)(q13;q23) with FOXO4/AFX1. Translocation t(3;11)(q28;q23) with LPP. Translocation t(10;11)(q22;q23) with TET1. Translocation t(9;11)(q34;q23) with DAB2IP. Translocation t(4;11)(p12;q23) with FRYL. Fusion proteins KMT2A-MLLT1, KMT2A-MLLT3 and KMT2A-ELL interact with PPP1R15A and, on the contrary to unfused KMT2A, inhibit PPP1R15A-induced apoptosis.",10490642
2842,Q03164,,,Note=A chromosomal aberration involving KMT2A may be a cause of chronic neutrophilic leukemia. Translocation t(4;11)(q21;q23) with SEPT11.,10490642


In [12]:
df_U_disease[df_U_disease.Entry_U=='Q03164']

,Entry_U,InvolmentInDisease_U
10249,Q03164,"DISEASE: Wiedemann-Steiner syndrome (WDSTS) [MIM:605130]: A syndrome characterized by hairy elbows (hypertrichosis cubiti), intellectual disability, a distinctive facial appearance, and short stature. Facial characteristics include long eyelashes, thick or arched eyebrows with a lateral flare, and downslanting and vertically narrow palpebral fissures. {ECO:0000269|PubMed:22795537}. Note=The disease is caused by mutations affecting the gene represented in this entry.; DISEASE: Note=Chromosomal aberrations involving KMT2A are a cause of acute leukemias. Translocation t(1;11)(q21;q23) with MLLT11/AF1Q; translocation t(3;11)(p21;q23) with NCKIPSD/AF3p21; translocation t(3,11)(q25,q23) with GMPS; translocation t(4;11)(q21;q23) with AFF1/MLLT2/AF4; insertion ins(5;11)(q31;q13q23) with AFF4/AF5Q31; translocation t(5;11)(q12;q23) with AF5-alpha/CENPK; translocation t(6;11)(q27;q23) with AFDN; translocation t(9;11)(p22;q23) with MLLT3/AF9; translocation t(10;11)(p11.2;q23) with ABI1; translocation t(10;11)(p12;q23) with MLLT10/AF10; t(11;15)(q23;q14) with KNL1 and ZFYVE19; translocation t(11;17)(q23;q21) with MLLT6/AF17; translocation t(11;19)(q23;p13.3) with ELL; translocation t(11;19)(q23;p13.3) with MLLT1/ENL; translocation t(11;19)(q23;p23) with GAS7; translocation t(X;11)(q13;q23) with FOXO4/AFX1. Translocation t(3;11)(q28;q23) with LPP. Translocation t(10;11)(q22;q23) with TET1. Translocation t(9;11)(q34;q23) with DAB2IP. Translocation t(4;11)(p12;q23) with FRYL. Fusion proteins KMT2A-MLLT1, KMT2A-MLLT3 and KMT2A-ELL interact with PPP1R15A and, on the contrary to unfused KMT2A, inhibit PPP1R15A-induced apoptosis. {ECO:0000269|PubMed:10490642}.; DISEASE: Note=A chromosomal aberration involving KMT2A may be a cause of chronic neutrophilic leukemia. Translocation t(4;11)(q21;q23) with SEPT11. {ECO:0000269|PubMed:10490642}."


In [13]:
_df_D[_df_D.duplicated(keep=False)]

,Entry_U,Disease_D,MIM_D,Desc_D,PubMed_D


In [14]:
df_U[df_U.Entry_U=='P26367']

GeneName Entry_U                                                       ProteinName_U            Organism_U Entryname_U  \
125     PAX6  P26367  Paired box protein Pax-6 (Aniridia type II protein) (Oculorhombin)  Homo sapiens (Human)  PAX6_HUMAN   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [15]:
_df_D[_df_D.Entry_U=='P26367']

,Entry_U,Disease_D,MIM_D,Desc_D,PubMed_D
38,P26367,Aniridia 1 (AN1),106210,"Aniridia 1 (AN1) [MIM:106210]: A congenital, bilateral, panocular disorder characterized by complete absence of the iris or extreme iris hypoplasia. Aniridia is not just an isolated defect in iris development but it is associated with macular and optic nerve hypoplasia, cataract, corneal changes, nystagmus. Visual acuity is generally low but is unrelated to the degree of iris hypoplasia. Glaucoma is a secondary problem causing additional visual loss over time.","10234503,10737978,11309364,11553050,11826019,12552561,12634864,16493447,17595013,21850189,24033328,8364574,9147640,9281415,9792406,9856761,9931324,,"
39,P26367,Anterior segment dysgenesis 5 (ASGD5),604229,"Anterior segment dysgenesis 5 (ASGD5) [MIM:604229]: A form of anterior segment dysgenesis, a group of defects affecting anterior structures of the eye including cornea, iris, lens, trabecular meshwork, and Schlemm canal. Anterior segment dysgeneses result from abnormal migration or differentiation of the neural crest derived mesenchymal cells that give rise to components of the anterior chamber during eye development. Different anterior segment anomalies may exist alone or in combination, including iris hypoplasia, enlarged or reduced corneal diameter, corneal vascularization and opacity, posterior embryotoxon, corectopia, polycoria, abnormal iridocorneal angle, ectopia lentis, and anterior synechiae between the iris and posterior corneal surface. Clinical conditions falling within the phenotypic spectrum of anterior segment dysgeneses include aniridia, Axenfeld anomaly, Reiger anomaly/syndrome, Peters anomaly, and iridogoniodysgenesis.","10441571,12721955,8162071"
40,P26367,Foveal hypoplasia 1 (FVH1),136520,"Foveal hypoplasia 1 (FVH1) [MIM:136520]: An isolated form of foveal hypoplasia, a developmental defect of the eye defined as the lack of foveal depression with continuity of all neurosensory retinal layers in the presumed foveal area. Clinical features include absence of foveal pit on optical coherence tomography, absence of foveal hyperpigmentation, absence of foveal avascularity, absence of foveal and macular reflexes, decreased visual acuity, and nystagmus. Anterior segment anomalies and cataract are observed in some FVH1 patients.","8640214,9931324"
41,P26367,Keratitis hereditary (KERH),148190,"Keratitis hereditary (KERH) [MIM:148190]: An ocular disorder characterized by corneal opacification, recurrent stromal keratitis and vascularization.",7668281
42,P26367,"Coloboma, ocular, autosomal dominant (COAD)",120200,"Coloboma, ocular, autosomal dominant (COAD) [MIM:120200]: A set of malformations resulting from abnormal morphogenesis of the optic cup and stalk, and the fusion of the fetal fissure (optic fissure). The clinical presentation is variable. Some individuals may present with minimal defects in the anterior iris leaf without other ocular defects. More complex malformations create a combination of iris, uveoretinal and/or optic nerve defects without or with microphthalmia or even anophthalmia.",12721955
43,P26367,Coloboma of optic nerve (COLON),120430,"Coloboma of optic nerve (COLON) [MIM:120430]: An ocular defect that is due to malclosure of the fetal intraocular fissure affecting the optic nerve head. In some affected individuals, it appears as enlargement of the physiologic cup with severely affected eyes showing huge cavities at the site of the disk.",12721955
44,P26367,Bilateral optic nerve hypoplasia (BONH),165550,"Bilateral optic nerve hypoplasia (BONH) [MIM:165550]: A congenital anomaly in which the optic disk appears abnormally small. It may be an isolated finding or part of a spectrum of anatomic and functional abnormalities that includes partial or complete agenesis of the septum pellucidum, other midline brain defects, cerebral anomalies, pituitary dysfunction, and structural abnormalities of the pituitary.",12721955
45,P26367,Aniridia 2 (AN2),617141,"Aniridia 2 (AN2)

In [16]:
print(_df_V.shape,_df_D.shape)

(55859, 7) (6116, 5)


In [17]:
_df_V.sample(2)

,Entry_U,VariantID_V,VariantSource_U,MutationDescription_V,dbSNP_V,Mutation_V,PubMed_V
25066,P42701,VAR_021285,Uniprot,Substitution-Missense,rs17884957,339 339 H -> Q,
28808,O94972,VAR_060218,Uniprot,Substitution-Missense,rs17853504,108 108 T -> A,15489334


In [18]:
_df_D[_df_D.PubMed_D.isin(_df_V.PubMed_V)]

,Entry_U,Disease_D,MIM_D,Desc_D,PubMed_D
3,Q96JQ0,Van Maldergem syndrome 1 (VMLDS1),601390,"Van Maldergem syndrome 1 (VMLDS1) [MIM:601390]: An autosomal recessive disorder characterized by intellectual disability, typical craniofacial features, auditory malformations resulting in hearing loss, and skeletal and limb malformations. Some patients have renal hypoplasia. Brain MRI typically shows periventricular nodular heterotopia.",24056717
4,Q96JQ0,Mitral valve prolapse 2 (MVP2),607829,"Mitral valve prolapse 2 (MVP2) [MIM:607829]: A form of mitral valve prolapse, a valvular hearth disease characterized by abnormally elongated and thickened mitral valve leaflets, that typically show myxomatous degeneration with increased leaflet compliance. It is associated with mitral regurgitation. Myxomatous mitral valves have an abnormal layered architecture characterized by loose collagen in fibrosa, expanded spongiosa strongly positive for proteoglycans, and disrupted elastin in atrialis. In classic mitral valve prolapse, leaflets are at least 5 mm thick, whereas in the non-classic form, they are less than 5 mm thick. Severe classic mitral valve prolapse is strongly associated with arrhythmias, endocarditis, heart failure, and need for valve surgery.",26258302
7,P35913,"Night blindness, congenital stationary, autosomal dominant 2 (CSNBAD2)",163500,"Night blindness, congenital stationary, autosomal dominant 2 (CSNBAD2) [MIM:163500]: A non-progressive retinal disorder characterized by impaired night vision, often associated with nystagmus and myopia.",8075643
9,P51160,Achromatopsia 5 (ACHM5),613093,"Achromatopsia 5 (ACHM5) [MIM:613093]: A form of achromatopsia, an ocular stationary disorder due to the absence of functioning cone photoreceptors in the retina. It is characterized by total colorblindness, low visual acuity, photophobia and nystagmus. ACHM5 inheritance is autosomal recessive.","19615668,21127010,28583373"
10,Q15120,"Charcot-Marie-Tooth disease, X-linked dominant, 6 (CMTX6)",300905,"Charcot-Marie-Tooth disease, X-linked dominant, 6 (CMTX6) [MIM:300905]: A form of Charcot-Marie-Tooth disease, a disorder of the peripheral nervous system, characterized by progressive weakness and atrophy, initially of the peroneal muscles and later of the distal muscles of the arms. Charcot-Marie-Tooth disease is classified in two main groups on the basis of electrophysiologic properties and histopathology: primary peripheral demyelinating neuropathies characterized by severely reduced motor nerve conduction velocities (NCVs) (less than 38m/s) and segmental demyelination and remyelination, and primary peripheral axonal neuropathies characterized by normal or mildly reduced NCVs and chronic axonal degeneration and regeneration on nerve biopsy.",23297365
11,Q9P0J1,Pyruvate dehydrogenase phosphatase deficiency (PDP deficiency),608782,Pyruvate dehydrogenase phosphatase deficiency (PDP deficiency) [MIM:608782]: Results in lactic acidosis leading to neurological dysfunction.,15855260
22,P08575,Severe combined immunodeficiency autosomal recessive T-cell-negative/B-cell-positive/NK-cell-positive (T(-)B(+)NK(+) SCID),608971,"Severe combined immunodeficiency autosomal recessive T-cell-negative/B-cell-positive/NK-cell-positive (T(-)B(+)NK(+) SCID) [MIM:608971]: A form of severe combined immunodeficiency (SCID), a genetically and clinically heterogeneous group of rare congenital disorders characterized by impairment of both humoral and cell-mediated immunity, leukopenia, and low or absent antibody levels. Patients present in infancy recurrent, persistent infections by opportunistic organisms. The common characteristic of all types of SCID is absence of T-cell-mediated cellular immunity due to a defect in T-cell development.",11145714
27,P04637,Adrenocortical carcinoma (ADCC),202300,Adrenocortical carcinoma (ADCC) [MIM:202300]: A malignant neoplasm of the adrenal cortex and a rare childhood tumor. It occurs with increased frequency in patients with Beckwith-Wiede

In [19]:
_df_V[_df_V.Entry_U=='P51160']

,Entry_U,VariantID_V,VariantSource_U,MutationDescription_V,dbSNP_V,Mutation_V,PubMed_V
46,P51160,VAR_062408,Uniprot,Substitution-Missense,rs121918537,29 29 R -> W,"19615668,21127010,28583373"
47,P51160,VAR_079307,Uniprot,Substitution-Missense,rs769506319,104 104 R -> W,"21127010,28583373"
48,P51160,VAR_025470,Uniprot,Substitution-Missense,rs76999928,157 157 D -> E,10393054
49,P51160,VAR_050475,Uniprot,Substitution-Missense,rs701865,270 270 S -> T,8641425
50,P51160,VAR_079308,Uniprot,Substitution-Missense,,276 858 Missing,21127010
51,P51160,VAR_062409,Uniprot,Substitution-Missense,rs121918538,323 323 Y -> N,"19615668,21127010,28583373"
52,P51160,VAR_079309,Uniprot,Substitution-Missense,,391 391 P -> L,"21127010,28583373"
53,P51160,VAR_062410,Uniprot,Substitution-Missense,rs121918539,455 455 M -> V,"19615668,28583373"
54,P51160,VAR_079310,Uniprot,Substitution-Missense,rs267606934,602 602 H -> L,"21127010,28583373"
55,P51160,VAR_079311,Uniprot,Substitution-Missense,rs267606936,790 790 E -> K,"21127010,28583373"


In [20]:
_df_VD=pd.merge(_df_V,_df_D,how='left', left_on=['Entry_U','PubMed_V'], right_on =['Entry_U','PubMed_D']) #on=['Entry_U','PubMed'])
_df_VD.shape

(56189, 11)

In [21]:
_df_VD[_df_VD.Entry_U=='Q93074']

,Entry_U,VariantID_V,VariantSource_U,MutationDescription_V,dbSNP_V,Mutation_V,PubMed_V,Disease_D,MIM_D,Desc_D,PubMed_D
48153,Q93074,VAR_033112,Uniprot,Substitution-Missense,rs80338758,961 961 R -> W,17334363,Opitz-Kaveggia syndrome (OKS),305450,"Opitz-Kaveggia syndrome (OKS) [MIM:305450]: X-linked disorder characterized by mental retardation, relative macrocephaly, hypotonia and constipation.",17334363
48154,Q93074,VAR_037534,Uniprot,Substitution-Missense,rs80338759,1007 1007 N -> S,17369503,Lujan-Fryns syndrome (LUJFRYS),309520,"Lujan-Fryns syndrome (LUJFRYS) [MIM:309520]: Clinically, Lujan-Fryns syndrome can be distinguished from Opitz-Kaveggia syndrome by tall stature, hypernasal voice, hyperextensible digits and high nasal root.",17369503
48155,Q93074,VAR_069770,Uniprot,Substitution-Missense,rs387907360,1148 1148 R -> H,23395478,"Ohdo syndrome, X-linked (OHDOX)",300895,"Ohdo syndrome, X-linked (OHDOX) [MIM:300895]: A syndrome characterized by mental retardation, feeding problems, and distinctive facial appearance with coarse facial features, severe blepharophimosis, ptosis, a bulbous nose, micrognathia and a small mouth. Dental hypoplasia and deafness can be considered as common manifestations of the syndrome. Male patients show cryptorchidism and scrotal hypoplasia.",23395478
48156,Q93074,VAR_069771,Uniprot,Substitution-Missense,rs387907361,1165 1165 S -> P,23395478,"Ohdo syndrome, X-linked (OHDOX)",300895,"Ohdo syndrome, X-linked (OHDOX) [MIM:300895]: A syndrome characterized by mental retardation, feeding problems, and distinctive facial appearance with coarse facial features, severe blepharophimosis, ptosis, a bulbous nose, micrognathia and a small mouth. Dental hypoplasia and deafness can be considered as common manifestations of the syndrome. Male patients show cryptorchidism and scrotal hypoplasia.",23395478
48157,Q93074,VAR_046672,Uniprot,Substitution-Missense,rs1139013,1392 1392 Q -> R,"10198638,8724849",NaN,NaN,NaN,NaN
48158,Q93074,VAR_069772,Uniprot,Substitution-Missense,rs387907362,1729 1729 H -> N,23395478,"Ohdo syndrome, X-linked (OHDOX)",300895,"Ohdo syndrome, X-linked (OHDOX) [MIM:300895]: A syndrome characterized by mental retardation, feeding problems, and distinctive facial appearance with coarse facial features, severe blepharophimosis, ptosis, a bulbous nose, micrognathia and a small mouth. Dental hypoplasia and deafness can be considered as common manifestations of the syndrome. Male patients show cryptorchidism and scrotal hypoplasia.",23395478
48159,Q93074,VAR_074018,Uniprot,Substitution-Missense,rs879255528,1974 1974 Q -> H,26273451,NaN,NaN,NaN,NaN


In [22]:
df_U_mod=df_U[['GeneName','Entry_U','ProteinName_U','Organism_U','Entryname_U']]
df_U_mod.shape

(20262, 5)

In [23]:
df_merged_U=pd.merge(_df_VD,df_U_mod,how='left', on=['Entry_U'])
df_merged_U.shape

(56189, 15)

In [24]:
df_merged_U.sample(21)

,Entry_U,VariantID_V,VariantSource_U,MutationDescription_V,dbSNP_V,Mutation_V,PubMed_V,Disease_D,MIM_D,Desc_D,PubMed_D,GeneName,ProteinName_U,Organism_U,Entryname_U
237,O95340,VAR_073027,Uniprot,Substitution-Missense,,76 76 L -> Q,23824674,NaN,NaN,NaN,NaN,PAPSS2,Bifunctional 3'-phosphoadenosine 5'-phosphosulfate synthase 2 (PAPS synthase 2) (PAPSS 2) (Sulfurylase kinase 2) (SK 2) (SK2) [Includes: Sulfate adenylyltransferase (EC 2.7.7.4) (ATP-sulfurylase) (Sulfate adenylate transferase) (SAT); Adenylyl-sulfate kinase (EC 2.7.1.25) (3'-phosphoadenosine-5'-phosphosulfate synthase) (APS kinase) (Adenosine-5'-phosphosulfate 3'-phosphotransferase) (Adenylylsulfate 3'-phosphotransferase)],Homo sapiens (Human),PAPS2_HUMAN
22300,Q15928,VAR_069637,Uniprot,Substitution-Missense,rs587776959,474 474 T -> I,23160277,"Polydactyly, postaxial A6 (PAPA6)",615226,"Polydactyly, postaxial A6 (PAPA6) [MIM:615226]: A condition characterized by the occurrence of supernumerary digits in the upper and/or lower extremities. In postaxial polydactyly type A, the extra digit is well-formed and articulates with the fifth or a sixth metacarpal/metatarsal.",23160277,ZNF141,Zinc finger protein 141,Homo sapiens (Human),ZN141_HUMAN
48541,Q13651,VAR_016294,Uniprot,Substitution-Missense,rs4252250,61 61 L -> V,,NaN,NaN,NaN,NaN,IL10RA,Interleukin-10 receptor subunit alpha (IL-10 receptor subunit alpha) (IL-10R subunit alpha) (IL-10RA) (CDw210a) (Interleukin-10 receptor subunit 1) (IL-10R subunit 1) (IL-10R1) (CD antigen CD210),Homo sapiens (Human),I10R1_HUMAN
44376,P04150,VAR_075803,Uniprot,Substitution-Missense,,726 726 H -> R,26031419,NaN,NaN,NaN,NaN,NR3C1,Glucocorticoid receptor (GR) (Nuclear receptor subfamily 3 group C member 1),Homo sapiens (Human),GCR_HUMAN
35265,Q8WZ42,VAR_069432,Uniprot,Substitution-Missense,rs780672348,16046 16046 G -> V,23033978,NaN,NaN,NaN,NaN,TTN,Titin (EC 2.7.11.1) (Connectin) (Rhabdomyosarcoma antigen MU-RMS-40.14),Homo sapiens (Human),TITIN_HUMAN
26812,Q04671,VAR_006132,Uniprot,Substitution-Missense,rs121918166,443 443 V -> I,"10987646,12876664,17385796",NaN,NaN,NaN,NaN,OCA2,P protein (Melanocyte-specific transporter protein) (Pink-eyed dilution protein homolog),Homo sapiens (Human),P_HUMAN
24072,Q9UHC1,VAR_023349,Uniprot,Substitution-Missense,rs175080,844 844 P -> L,"10615123,",NaN,NaN,NaN,NaN,MLH3,DNA mismatch repair protein Mlh3 (MutL protein homolog 3),Homo sapiens (Human),MLH3_HUMAN
13642,P11362,VAR_030973,Uniprot,Substitution-Missense,rs765615419,129 129 D -> A,15605412,NaN,NaN,NaN,NaN,FGFR1,Fibroblast growth factor receptor 1 (FGFR-1) (EC 2.7.10.1) (Basic fibroblast growth factor receptor 1) (BFGFR) (bFGF-R-1) (Fms-like tyrosine kinase 2) (FLT-2) (N-sam) (Proto-oncogene c-Fgr) (CD antigen CD331),Homo sapiens (Human),FGFR1_HUMAN
50251,P06865,VAR_003230,Uniprot,Substitution-Missense,,335 335 I -> F,7951261,NaN,NaN,NaN,NaN,HEXA,Beta-hexosaminidase subunit alpha (EC 3.2.1.52) (Beta-N-acetylhexosaminidase subunit alpha) (Hexosaminidase subunit A) (N-acetyl-beta-glucosaminidase subunit alpha),Homo sapiens (Human),HEXA_HUMAN
8469,O95255,VAR_067883,Uniprot,Substitution-Missense,rs61340537,946 946 L -> I,"16086317,19339160",NaN,NaN,NaN,NaN,ABCC6,Multidrug resistance-associated protein 6 (ATP-binding cassette sub-family C member 6) (Anthracycline resistance-associated protein) (Multi-specific organic anion transporter E) (MOAT-E),Homo sapiens (Human),MRP6_HUMAN


In [25]:
df_merged_U.to_csv("Uniprot_cleaned.csv",sep="^")

# Number of entires in above Uniprot list after removing entries without gene name: 56,189 with 15 columns.

# Processing GWAS

In [ ]:
blob = storage.Blob('genetics/gwas_catalog_v1.0.2-associations_e93_r2019-01-11.tsv',bucket)
with open('gwas_catalog_v1.0.2-associations_e93_r2019-01-11.tsv', 'wb') as file_obj:
    blob.download_to_file(file_obj)

In [26]:
#df_test=pd.read_csv('gwas_catalog_v1.0.2-associations_e93_r2019-01-11.tsv',sep='\t', header=0,low_memory=False)
chunksize = 10 ** 6
colnames=[]
frames=[]
GWAS=pd.DataFrame()
for chunkGWAS in pd.read_csv('gwas_catalog_v1.0.2-associations_e93_r2019-01-11.tsv',sep='\t', \
                         header=0,low_memory=False,chunksize=chunksize):
    GWAS=chunkGWAS[['STUDY ACCESSION','PUBMEDID','DISEASE/TRAIT',  'CHR_ID',\
                    'CHR_POS', 'REPORTED GENE(S)',\
                    'MAPPED_GENE', 'STRONGEST SNP-RISK ALLELE','CONTEXT',\
                    'P-VALUE','P-VALUE (TEXT)','OR or BETA', 'MAPPED_TRAIT']]
    print('checking if multiple chunks are present')

checking if multiple chunks are present


In [27]:
GWAS = GWAS.assign(VariantSource_GWAS='GWAS')
GWAS.columns

Index(['STUDY ACCESSION', 'PUBMEDID', 'DISEASE/TRAIT', 'CHR_ID', 'CHR_POS', 'REPORTED GENE(S)', 'MAPPED_GENE', 'STRONGEST SNP-RISK ALLELE', 'CONTEXT', 'P-VALUE', 'P-VALUE (TEXT)', 'OR or BETA', 'MAPPED_TRAIT', 'VariantSource_GWAS'], dtype='object')

In [28]:
GWAS.columns=['VariantID_GWAS', 'PubMed_GWAS', 'Disease_GWAS', 'CHR_ID_GWAS',\
 'CHR_POS_GWAS', 'GeneName_GWAS', 'Mapped_Gene_GWAS', 'dbSNP_GWAS',\
 'Mutation_GWAS', 'PValue_GWAS', 'PValue_Text_GWAS','OR_or_BETA_GWAS',\
 'Mapped_Trait_GWAS','VariantSource_GWAS']

In [29]:
GWAS.sample(2)

,VariantID_GWAS,PubMed_GWAS,Disease_GWAS,CHR_ID_GWAS,CHR_POS_GWAS,GeneName_GWAS,Mapped_Gene_GWAS,dbSNP_GWAS,Mutation_GWAS,PValue_GWAS,PValue_Text_GWAS,OR_or_BETA_GWAS,Mapped_Trait_GWAS,VariantSource_GWAS
31145,GCST000530,19915576,Parkinson's disease,12,40034759,LRRK2,SLC2A13,rs1994090-?,intron_variant,3E-8,NaN,1.3900,Parkinson's disease,GWAS
94781,GCST006442,30038396,Educational attainment (years of education),12,81864996,NaN,PPFIA2 - VN1R57P,rs11115056-A,intergenic_variant,2E-15,NaN,0.0187,self reported educational attainment,GWAS


In [30]:
#GWAS[["PubMed_GWAS", "OR_or_BETA_GWAS"]] = df[["PubMed_GWAS", "OR_or_BETA_GWAS"]].apply(pd.to)
GWAS[["PubMed_GWAS", "OR_or_BETA_GWAS"]] = GWAS[["PubMed_GWAS", "OR_or_BETA_GWAS"]].astype('str')
GWAS.dtypes

VariantID_GWAS        object
PubMed_GWAS           object
Disease_GWAS          object
CHR_ID_GWAS           object
CHR_POS_GWAS          object
GeneName_GWAS         object
Mapped_Gene_GWAS      object
dbSNP_GWAS            object
Mutation_GWAS         object
PValue_GWAS           object
PValue_Text_GWAS      object
OR_or_BETA_GWAS       object
Mapped_Trait_GWAS     object
VariantSource_GWAS    object
dtype: object

In [31]:
GWAS.to_csv("GWAS_cleaned.csv")

# Processing Clinvar

In [ ]:
blob = storage.Blob('genetics/variant_summary.txt.gz',bucket)
with open('variant_summary.txt.gz', 'wb') as file_obj:
    blob.download_to_file(file_obj)

In [88]:
import pandas as pd
chunksize = 10 ** 6
Clinvar_V=pd.DataFrame()
for chunk in pd.read_csv('variant_summary.txt.gz',sep='\t',header=0,\
                         low_memory=False,chunksize=chunksize):
    Clinvar_V=chunk[['#AlleleID', 'Type', 'Name', 'GeneSymbol','GeneID',\
                     'ClinicalSignificance', 'RS# (dbSNP)','Origin',\
                     'Assembly']].loc[chunk.Assembly=='GRCh38']
    print('checking if multiple chunks are present')

checking if multiple chunks are present


In [89]:
Clinvar_V.shape

(458901, 9)

In [90]:
Clinvar_V.columns

Index(['#AlleleID', 'Type', 'Name', 'GeneSymbol', 'GeneID', 'ClinicalSignificance', 'RS# (dbSNP)', 'Origin', 'Assembly'], dtype='object')

In [91]:
Clinvar_V.sample(1)

,#AlleleID,Type,Name,GeneSymbol,GeneID,ClinicalSignificance,RS# (dbSNP),Origin,Assembly
456344,315487,single nucleotide variant,NM_000124.3(ERCC6):c.*1383T>G,ERCC6,2074,Uncertain significance,886047023,germline,GRCh38


In [92]:
Clinvar_V.columns=['VariantID_CVAR', 'MutationDescription_CVAR', 'Mutation_CVAR', 'GeneName_CVAR','GeneID_CVAR',\
                   'ClinicalSignificance_CVAR', 'dbSNP_CVAR','Origin_CVAR', 'Assembly_CVAR']

In [93]:
Clinvar_V=Clinvar_V.assign(VariantSource_CVAR='CVAR')

In [94]:
Clinvar_V.sample(1)

,VariantID_CVAR,MutationDescription_CVAR,Mutation_CVAR,GeneName_CVAR,GeneID_CVAR,ClinicalSignificance_CVAR,dbSNP_CVAR,Origin_CVAR,Assembly_CVAR,VariantSource_CVAR
137928,138220,single nucleotide variant,NM_001077490.2(GNAS):c.1240C>T (p.Pro414Ser),GNAS,2778,not provided,587778383,germline,GRCh38,CVAR


In [39]:
#Clinvar_V.to_csv("Clinvar_cleaned.csv") # bugs found pls check testing section

# Testing CVAR dataset

In [122]:
#Clinvar_V[Clinvar_V.VariantID_CVAR== 40621]
#Clinvar_V[Clinvar_V.VariantID_CVAR== 163880]
#Clinvar_V[Clinvar_V.VariantID_CVAR== 72480]
Clinvar_V[Clinvar_V.VariantID_CVAR== 20902]

,VariantID_CVAR,MutationDescription_CVAR,Mutation_CVAR,GeneName_CVAR,GeneID_CVAR,ClinicalSignificance_CVAR,dbSNP_CVAR,Origin_CVAR,Assembly_CVAR,VariantSource_CVAR
10551,20902,single nucleotide variant,NM_006640.4(SEPT9):c.262C>T (p.Arg88Trp),SEPT9,10801,Pathogenic,80338761,germline,GRCh38,CVAR


In [96]:
Clinvar_V['GeneName_CVAR']=Clinvar_V['GeneName_CVAR'].apply(lambda x:x.split(':')[1] if len(x.split(':'))>1 else x)

In [115]:
def fixGeneName(row):
    if len(row.split(';'))>0:
        for x in row.split(';'):
            if x[:3] not in ['C10','LIN','LOC']:
                return x                
    return

In [113]:
Clinvar_V['GeneName_CVAR']=Clinvar_V['GeneName_CVAR'].apply(lambda x: fixGeneName(x))

In [116]:
Clinvar_V.sample(3)

,VariantID_CVAR,MutationDescription_CVAR,Mutation_CVAR,GeneName_CVAR,GeneID_CVAR,ClinicalSignificance_CVAR,dbSNP_CVAR,Origin_CVAR,Assembly_CVAR,VariantSource_CVAR
657558,432329,single nucleotide variant,NM_003041.3(SLC5A2):c.1451G>A (p.Gly484Asp),SLC5A2,6524,Likely pathogenic,1309307492,germline,GRCh38,CVAR
455758,315194,single nucleotide variant,NM_000503.5(EYA1):c.*848G>A,EYA1,2138,Uncertain significance,775498240,germline,GRCh38,CVAR
713955,462983,single nucleotide variant,NM_000059.3(BRCA2):c.950C>T (p.Thr317Ile),BRCA2,675,Uncertain significance,1341473353,germline,GRCh38,CVAR


In [121]:
Clinvar_V.to_csv("Clinvar_cleaned.csv")

# Processing COSMIC

#### Note: Import mutation dataset from COSMIC database. It provides a tab separated table of the complete curated COSMIC dataset (targeted screens) from the current release. It includes all coding point mutations, and the negative data set. The name of the file is 'CosmicCompleteTargetedScreensMutantExport.tsv.gz'. 
#### We will only consider those entires which have Mutation IDs.

In [ ]:
blob = storage.Blob('genetics/CosmicCompleteTargetedScreensMutantExport.tsv',bucket)
with open('CosmicCompleteTargetedScreensMutantExport.tsv', 'wb') as file_obj:
    blob.download_to_file(file_obj)

In [40]:
chunksize = 10 ** 6
cosmic_C=pd.DataFrame()
colnames=["GeneName","AccessionNumber_C","GeneCDSlength","HGNCid","SampleName","SampleId", \
          "IdTumour", "PrimarySite","SiteSubtype1","SiteSubtype2","SiteSubtype3", \
          "PrimaryHistology","HistologySubtype1", "HistologySubtype2","HistologySubtype3", \
          "GenomeWideScreen","MutationId_C","MutationCDS","MutationAA_C",\
          "MutationDescription_C","MutationZygosity","LOH","GRCh",\
          "MutationGenomePosition_C","MutationStrand","SNP","ResistanceMutation",\
          "FATHMMPrediction","FATHMMScore","MutationSomaticStatus","Pubmed_PMID_C",\
          "IdStudy","SampleSource","TumourOrigin","Age"]
frames=[]

for chunk in pd.read_csv('CosmicCompleteTargetedScreensMutantExport.tsv',sep='\t',header=0, \
                         names=colnames,low_memory=False, \
                 dtype={"GeneName":object,"AccessionNumber":object,\
                        "GeneCDSlength":object,"HGNCid":object,\
                        "SampleName":object,"SampleId":object,\
                        "IdTumour":object,"PrimarySite":object,\
                        "SiteSubtype1":object,"SiteSubtype2":object, \
                        "SiteSubtype3":object,"PrimaryHistology":object,\
                        "HistologySubtype1":object,"HistologySubtype2":object,\
                        "HistologySubtype3":object,"GenomeWideScreen":object, \
                        "MutationId":object,"MutationCDS":object,"MutationAA_C":object, \
                        "MutationDescription":object, "MutationZygosity":object,
                        "LOH":object,"GRCh":object,"MutationGenomePosition":object,
                        "MutationStrand":object,"SNP":object,"ResistanceMutation":object,\
                        "FATHMMPrediction":object,"FATHMMScore":object,\
                        "MutationSomaticStatus":object,"Pubmed_PMID_C":object, \
                        "IdStudy":object,"SampleSource":object,"TumourOrigin":object,\
                        "Age":object}, \
                 chunksize=chunksize
                ):
    cosmic_C=chunk[['GeneName','MutationId_C','MutationDescription_C' \
                    ,'MutationAA_C','Pubmed_PMID_C']].loc[chunk.MutationId_C.notnull()]
    frames.append(cosmic_C)
    print('checking number of chunks')
                  #selecting only with mutation id from Cosmic file

checking number of chunks
checking number of chunks
checking number of chunks
checking number of chunks
checking number of chunks
checking number of chunks
checking number of chunks
checking number of chunks
checking number of chunks
checking number of chunks
checking number of chunks
checking number of chunks


In [41]:
if len(frames)>1:
    cosmic_merged = pd.concat(frames)

In [42]:
len(frames),cosmic_C.shape,cosmic_merged.shape

(12, (34165, 5), (647327, 5))

In [43]:
cosmic_merged.sample(2)

,GeneName,MutationId_C,MutationDescription_C,MutationAA_C,Pubmed_PMID_C
2236879,CDKN2A,COSM12526,Whole gene deletion,p.0,18799215
1440150,KRAS,COSM521,Substitution - Missense,p.G12D,21894049


In [44]:
cosmic_merged=cosmic_merged.assign(VariantSource_COSMIC='COSMIC')

In [45]:
cosmic_merged.columns=['GeneName_COSMIC','VariantID_COSMIC', 'MutationDescription_COSMIC',\
                      'Mutation_COSMIC','PubMed_COSMIC','VariantSource']

In [46]:
cosmic_merged.sample(2)

,GeneName_COSMIC,VariantID_COSMIC,MutationDescription_COSMIC,Mutation_COSMIC,PubMed_COSMIC,VariantSource
4593511,KRAS,COSM520,Substitution - Missense,p.G12V,25986173,COSMIC
537387,EGFR,COSM13243,Deletion - In frame,p.(729_761)del?,22115706,COSMIC


## COSMIC dataset contains duplicate rows.

In [47]:
cosmic_merged[(cosmic_merged.VariantID_COSMIC=='COSM12979') & (cosmic_merged.PubMed_COSMIC=='21681971')]

,GeneName_COSMIC,VariantID_COSMIC,MutationDescription_COSMIC,Mutation_COSMIC,PubMed_COSMIC,VariantSource
5501371,EGFR,COSM12979,Substitution - Missense,p.L858R,21681971,COSMIC
11554738,EGFR,COSM12979,Substitution - Missense,p.L858R,21681971,COSMIC


In [48]:
cosmic_merged.shape

(647327, 6)

In [49]:
cosmic_merged=cosmic_merged.drop_duplicates(keep='first')
cosmic_merged.shape

(284281, 6)

In [50]:
647327-284281

363046

## Please note that we have 36,3046 rows as duplicates.

## Merging all PubMed ids under one row using a group by function.

In [51]:
groupbyName = cosmic_merged.columns.difference(['PubMed_COSMIC']).tolist()
groupbyName

['GeneName_COSMIC',
 'MutationDescription_COSMIC',
 'Mutation_COSMIC',
 'VariantID_COSMIC',
 'VariantSource']

In [52]:
cosmic_final=cosmic_merged.astype(str).groupby(groupbyName, as_index=False).PubMed_COSMIC.agg(','.join)

In [53]:
cosmic_final[(cosmic_final.VariantID_COSMIC=='COSM12979')]

GeneName_COSMIC MutationDescription_COSMIC Mutation_COSMIC VariantID_COSMIC VariantSource  \
50476            EGFR    Substitution - Missense         p.L858R        COSM12979        COSMIC   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [ ]:
#s = ['p.L858R','p.C229fs*10','p.?_?ins?','p.E604_F605ins15','p.EAC746_E749delELRE','p.D770_N771insSVD','p.?fs*?']
#a=[re.findall(r"[^\W\d_]+|\d+",i[1:]) for i in s]
#a

In [54]:
cosmic_final.sample(3)

,GeneName_COSMIC,MutationDescription_COSMIC,Mutation_COSMIC,VariantID_COSMIC,VariantSource,PubMed_COSMIC
139046,PIK3CG,Substitution - Nonsense,p.Q533*,COSM6968654,COSMIC,28481359
180849,TCF7L2_ENST00000355717,Substitution - Missense,p.S72F,COSM6474579,COSMIC,28002797
978,ABL1_ENST00000318560,Substitution - Missense,p.G768R,COSM6927344,COSMIC,28481359


In [55]:
#cosmic_final.to_csv("COSMIC_cleaned.csv") - bugs found check testing section

# Testing COSMIC

In [117]:
cosmic_final[(cosmic_final.VariantID_COSMIC=='COSM5027226')]

,GeneName_COSMIC,MutationDescription_COSMIC,Mutation_COSMIC,VariantID_COSMIC,VariantSource,PubMed_COSMIC
3,A1CF_ENST00000282641,Substitution - Missense,p.G410A,COSM5027226,COSMIC,22722193


In [118]:
cosmic_final['GeneName_COSMIC']=cosmic_final['GeneName_COSMIC'].apply(lambda x:x.split('_')[0] if len(x.split('_'))>1 else x)

In [119]:
cosmic_final[(cosmic_final.VariantID_COSMIC=='COSM5027226')]

,GeneName_COSMIC,MutationDescription_COSMIC,Mutation_COSMIC,VariantID_COSMIC,VariantSource,PubMed_COSMIC
3,A1CF,Substitution - Missense,p.G410A,COSM5027226,COSMIC,22722193


In [120]:
cosmic_final.to_csv("COSMIC_cleaned.csv")

# Merging all databases

In [56]:
df_merged_U.shape,cosmic_final.shape,Clinvar_V.shape, GWAS.shape

((56189, 15), (212006, 6), (458901, 10), (104767, 14))

In [ ]:
df=pd.merge(df_merged_U,GWAS,how='left', left_on=['GeneName'], right_on =['GeneName_GWAS'])
df.shape

In [ ]:
df.to_csv("df.csv",index=None)
Clinvar_V.to_csv("CVAR.csv",index=None)
cosmic_final.to_csv("COSMIC.csv",index=None)

In [ ]:
df_result = pd.DataFrame(columns=(df.columns.append(Clinvar_V.columns)).unique())

In [ ]:
df_result.columns

In [ ]:
df_result.to_csv("temp.csv",index_label=False)

In [ ]:
def preprocess(x):
    df2=pd.merge(df,x,how='left', left_on=['GeneName'], right_on =['GeneName_CVAR'])
    df2.to_csv("temp.csv",mode="a",header=False,index=False)

In [ ]:
#reader = pd.read_csv("CVAR.csv", chunksize=1000)
#[preprocess(r) for r in reader]

In [ ]:
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
   ...:                     'B': ['B0', 'B1', 'B2', 'B3'],
   ...:                     'C': ['C0', 'C1', 'C2', 'C3'],
   ...:                     'D': ['D0', 'D1', 'D2', 'D3']},
   ...:                    index=[0, 1, 2, 3])

In [ ]:
df1

In [ ]:
df2 = pd.DataFrame({'A1': ['A0', 'A5', 'A1', 'A7'],
   ...:                     'B1': ['B4', 'B5', 'B6', 'B7'],
   ...:                     'C1': ['C4', 'C5', 'C6', 'C7'],
   ...:                     'D1': ['D4', 'D5', 'D6', 'D7']},
   ...:                    index=[4, 5, 6, 7])

In [ ]:
df2

In [ ]:
df2.to_csv('df22.csv',index=None)

In [ ]:
df_result = pd.DataFrame(columns=(df1.columns.append(df2.columns)).unique())

In [ ]:
df_result.columns

In [ ]:
df_result.to_csv("temp.csv",index_label=False)

In [ ]:
def preprocess(x):
    df2=pd.merge(df1,x,how='left', left_on=['A'], right_on =['A1'])
    df2.to_csv("temp.csv",mode="a",header=False,index=False)

In [ ]:
reader = pd.read_csv("df22.csv", chunksize=1)
[preprocess(r) for r in reader]

In [ ]:
pd.merge(df1,df2,how='outer', left_on=['A'], right_on =['A1'])

In [ ]:
df_merged_U.to_csv("UNIPROT.csv",index=None)
GWAS.to_csv("GWAS.csv",index=None)
Clinvar_V.to_csv("CVAR.csv",index=None)
cosmic_final.to_csv("COSMIC.csv",index=None)


In [1]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
df_U=pd.read_csv("UNIPROT.csv",header=0)
df_GWAS=pd.read_csv("GWAS.csv",header=0)
df_CVAR=pd.read_csv("CVAR.csv",header=0)
df_COSMIC=pd.read_csv("COSMIC.csv",header=0)

In [2]:
df=dd.merge(df_U,df_GWAS,how='outer', left_on=['GeneName'], right_on =['GeneName_GWAS'])
df.shape

(427925, 29)

In [8]:
Uniprot_Genes=df_U.GeneName.unique()
len(Uniprot_Genes)

9196

In [10]:
GWAS_Genes=df_GWAS.GeneName_GWAS.unique()
len(GWAS_Genes)

AttributeError: 'Series' object has no attribute 'strip'